In [ ]:
import os
os.chdir('../..')

In [116]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
import xgboost as xgb

In [12]:
input_path = r'C:\Users\Wojtek\Documents\Projects\MLPlayground\data\house_prices'
train_file = r'train.csv'
test_file = r'test.csv'

In [130]:
train = pd.read_csv(os.path.join(input_path, train_file))
test = pd.read_csv(os.path.join(input_path, test_file))

In [131]:
train.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [132]:
test.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


In [133]:
Ytr = train['SalePrice']
train.drop(['SalePrice'], axis=1, inplace=True)

In [134]:
whole = train.append(test)

In [135]:
imputer = KNNImputer()

In [136]:
whole_num = whole.select_dtypes(include=[np.number]).iloc[:,1:]

whole_str = whole.select_dtypes(include=[object]).iloc[:,1:]

In [137]:
whole_num = pd.DataFrame(data=imputer.fit_transform(whole_num), columns=whole_num.columns)

In [138]:
#imputer = KNNImputer()
imputer = SimpleImputer(strategy='most_frequent')

whole_str_imputed = pd.DataFrame(imputer.fit_transform(whole_str), columns=whole_str.columns)

In [148]:
whole_str_imputed = whole_str_imputed.apply(preprocessing.LabelEncoder().fit_transform)

In [150]:
whole = pd.concat([whole_num, whole_str_imputed], axis=1)

In [152]:
Xtr = whole.iloc[:train.shape[0], :]
Xte = whole.iloc[train.shape[0]:, :]

In [84]:
# Xtr['GarageYrBlt'].fillna(Xtr['GarageYrBlt'].median(), inplace=True)
# Xtr['MasVnrArea'].fillna(Xtr['MasVnrArea'].median(), inplace=True)
# Xtr['LotFrontage'].fillna(Xtr['LotFrontage'].median(), inplace=True)

C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [153]:
# Xte['GarageYrBlt'].fillna(Xte['GarageYrBlt'].median(), inplace=True)
# Xte['LotFrontage'].fillna(Xte['LotFrontage'].median(), inplace=True)
# Xte['MasVnrArea'].fillna(Xte['MasVnrArea'].median(), inplace=True)
# Xte['GarageArea'].fillna(Xte['GarageArea'].median(), inplace=True)

# Xte['BsmtFinSF1'].fillna(Xte['BsmtFinSF1'].median(), inplace=True)
# Xte['BsmtFinSF2'].fillna(Xte['BsmtFinSF2'].median(), inplace=True)
# Xte['BsmtUnfSF'].fillna(Xte['BsmtUnfSF'].median(), inplace=True)
# Xte['TotalBsmtSF'].fillna(Xte['TotalBsmtSF'].median(), inplace=True)

# Xte['BsmtFullBath'].fillna(Xte['BsmtFullBath'].median(), inplace=True)
# Xte['BsmtHalfBath'].fillna(Xte['BsmtHalfBath'].median(), inplace=True)

# Xte['GarageCars'].fillna(Xte['GarageCars'].median(), inplace=True)

In [154]:
Xtr.shape

(1460, 78)

In [155]:
Xte.shape

(1459, 78)

In [156]:
ss = StandardScaler().fit(Xtr)

Xtr = ss.transform(Xtr)
Xte = ss.transform(Xte)

In [100]:
classifier = SVC()

#classifier = LogisticRegression(max_iter=10000)
#classifier = RandomForestClassifier()
#classifier = GradientBoostingClassifier()
classifier.fit(Xtr, Ytr.values)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [101]:
result = classifier.predict(Xte)

In [157]:
model = xgb.XGBRegressor() 

In [158]:
model.fit(Xtr, Ytr.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [159]:
result = model.predict(Xte)

In [160]:
ids = test['Id'].values.astype('int')
output = np.vstack((ids, result))
output = pd.DataFrame(data=output.T, columns=['Id', 'SalePrice'])
output = output.astype('int')
output_path = r'C:\Users\Wojtek\Documents\Projects\MLPlayground\output'
output.to_csv(os.path.join(output_path, "house_prices.csv"), index=False)

In [ ]:
numerical = 

In [13]:
numerical_cols = ['MsSubCLass', 'LotFrontage', 'LotArea', '']



In [4]:
classifier = SVC()